# Detección de crecimiento urbano con imagenes del satélite Sentinel - 1

## 2_ Predicción

Este notebook genera la prediccion del modelo sobre un nuevo dataset. Para ello primero descargamos las imagenes satelitales y luego con satproc generamos el dataset de predicción con el formato necesario para el modelo de ML


**Definicion de los parametros para la predicción**

In [1]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import subprocess
import os

2022-06-02 16:43:40.005417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/cv2/../../lib64:
2022-06-02 16:43:40.005454: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
 
BANDS= "*"
YEARS =["2017","2021"]
BASE_PATH = "../"

#model
SIZE  = 100
STEP_SIZE= 100

### Descarga de las imagenes de predicción

Descarga las imagenes del bucket

In [3]:
def execute_download_images (YEAR,MODELO_AREA,PROVINCE):

    BUCKET = f"gs://dym-indunor-temp/immap/annotations_WASH/images/s1/{YEAR}/{MODELO_AREA}/{PROVINCE}/*_bands_{YEAR}*"

    img_folder = f'{BASE_PATH}/imagenes_s1_{MODELO_AREA}/{YEAR}/{PROVINCE}'
    !mkdir -p $img_folder
    !gsutil -m cp -r -n $BUCKET  $img_folder

### Satproc

Con esta herramienta genereamos un dataset de imágenes de predicción que, a diferencia del de entrenamiento, solo contiene imágenes.

In [4]:
def execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,YEAR,PROVINCE,MUNICIPIO):

    path_to_files  = f'{BASE_PATH}/imagenes_s1_{MODELO_AREA}/{YEAR}/{PROVINCE}/*.tif'
    dataset_folder_ = f'{BASE_PATH}/dataset_{MODELO_AREA}/data_predict/province_{PROVINCE}_{MUNICIPIO}/{YEAR}/{str(SIZE)}_{str(STEP_SIZE)}/'
    aoi_predict    = f'{BASE_PATH}/data/shp/aoi_pred/export_name_*_{PROVINCE}_{MUNICIPIO}_{MODELO_AREA}.gpkg'

    print("path_to_files-->",path_to_files)
    print("dataset_folder_-->",dataset_folder_)
    print("aoi_predict-->",aoi_predict)
    
    !satproc_extract_chips $path_to_files \
                    -o $dataset_folder_ \
                    --aoi $aoi_predict \
                    --size $SIZE \
                    --step-size $STEP_SIZE \
                    --rescale \
                    --rescale-mode values --min -15 --max 5
    return dataset_folder_

Pasamos la ruta de las imágenes de predicción y la ruta donde queremos que se guarde el resultado.



## Prediccion

Definimos la configuración para la predicción. Debemos pasar la ruta de las imágenes en el dataset de predicción, que es la ruta de salida de la línea anterior, y la correspondiente al modelo que entrenamos.

In [5]:
def excetute_prediction (dataset_folder,BASE_PATH,MODELO_AREA,PROVINCE,MUNICIPIO,YEAR,SIZE,STEP_SIZE):

    if MODELO_AREA == "pacifico": 
        MODEL = "UNet_160x160_spe100_3N_spe600_colombia_modelo4_vhvv2.h5"
 
    if MODELO_AREA == "atlantico": 
        MODEL = f'UNet_TEST_160x160_spe100_3N_spe100_colombia_atlantico.h5'
 
    
    predict_config = PredictConfig(
                                        images_path=os.path.join(dataset_folder), # ruta a las imagenes sobre las cuales queremos predecir
                                        results_path=os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results',f'province_{PROVINCE}_{MUNICIPIO}',YEAR,str(SIZE)+'_'+str(STEP_SIZE)), # ruta de destino para nuestra predicción
                                        batch_size=16,
                                        model_path=os.path.join(BASE_PATH +'data/weights', MODEL),  #  ruta al modelo (.h5)
                                        height=160,
                                        width=160,
                                        n_channels=3,
                                        n_classes=1,
                                        class_weights= [1])

    #A continuación ejecutamos la predicción
    predict(predict_config)    
    #plot_data_results(num_samples=2, fig_size=(2, 2), predict_config=predict_config, img_ch =2, n_bands=3)
    
    # join prediccions
    joined_folder = os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results',f'province_{PROVINCE}_{MUNICIPIO}',YEAR,str(SIZE)+'_'+str(STEP_SIZE)+'_joined/')
    srf_folder = predict_config.results_path+'/'                               
     
    print("size",SIZE)
    print("step size",STEP_SIZE)
        
    if STEP_SIZE==SIZE:
        !cp -r $srf_folder $joined_folder
    else:
        print("using ----> satproc_smooth_stitch")
        !satproc_smooth_stitch -o $joined_folder $srf_folder
        
    

    #create VTR
    !rm tmp_list.txt

    vrt_path        =  os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results/',f'province_{PROVINCE}_{MUNICIPIO}',YEAR,'all.vrt') # ruta de destino para nuestra predicción
    predict_results =  os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results/',f'province_{PROVINCE}_{MUNICIPIO}',YEAR,str(SIZE)+'_'+str(STEP_SIZE),'*tif') # ruta de destino para nuestra predicción

    print(predict_results)

    import glob
    filenames = glob.glob(predict_results)

    with open('tmp_list.txt', 'w') as f:
        for line in filenames:
            f.write(line)
            f.write('\n')

    !gdalbuildvrt -input_file_list tmp_list.txt $vrt_path
    
    # upload to a bucket
    BUCKET_PREDICTION_RESULTS = f'gs://dym-indunor-temp/immap/v3/results_test/resultados_UNET_pred/{MODELO_AREA}/{PROVINCE}/{MUNICIPIO}/{YEAR}/'

    all_results_folder = os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results',f'province_{PROVINCE}_{MUNICIPIO}',YEAR)
    !gsutil -m cp -r  $all_results_folder $BUCKET_PREDICTION_RESULTS
    

## RUN

In [8]:

import pandas as pd
  
# load the data with pd.read_csv
record = pd.read_csv('munis_v3.csv')

df=record.drop_duplicates(subset=['name', 'REGION'], keep='last')[['name', 'REGION']]
df = df.reset_index()  # make sure indexes pair with number of rows
print(df)
for index, row in df.iterrows():
   
    print(row['name'], row['REGION'])

    MODELO_AREA=row['REGION']
    PROVINCE=row['name']

    if PROVINCE=='ANTIOQUIA':
        #download images
        for YEAR in YEARS:   
            execute_download_images (YEAR,MODELO_AREA,PROVINCE)


        #download mountains masks 
        BUCKET_MOUNTAINS = f"gs://dym-indunor-temp/immap/annotations_WASH/images/DEM/{MODELO_AREA}/{PROVINCE}*"
        MOUNT_FOLDER = f"mountain_masks/{PROVINCE}/"
        ELEVATION_FILE = "mask_lt_25_.tif"
        mask_folder = BASE_PATH + MOUNT_FOLDER 
        !mkdir -p $mask_folder
        !gsutil -m cp -r $BUCKET_MOUNTAINS  $mask_folder
        !mv ../mountain_masks/$PROVINCE/$PROVINCE/mask_lt_25_.tif ../mountain_masks/$PROVINCE/

    index            name     REGION
0       5       atlantico  atlantico
1       8         bolivar  atlantico
2      11           cauca   pacifico
3      14           cesar  atlantico
4      16           choco   pacifico
5      24         cordoba  atlantico
6      28      la_guajira  atlantico
7      29       magdalena  atlantico
8      53          narino   pacifico
9      58           sucre  atlantico
10     76  valle_de_cauca   pacifico
11     77       ANTIOQUIA  atlantico
atlantico atlantico
bolivar atlantico
cauca pacifico
cesar atlantico
choco pacifico
cordoba atlantico
la_guajira atlantico
magdalena atlantico
narino pacifico
sucre atlantico
valle_de_cauca pacifico
ANTIOQUIA atlantico
Skipping existing item: file://..//imagenes_s1_atlantico/2017/ANTIOQUIA/VHd_VVd_VHdVVd_bands_2017_.tif
Copying gs://dym-indunor-temp/immap/annotations_WASH/images/s1/2021/atlantico/ANTIOQUIA/VHd_VVd_VHdVVd_bands_2021_.tif...
| [1/1 files][812.3 MiB/812.3 MiB] 100% Done  59.0 MiB/s ETA 00:00:00      

In [9]:
N=99
THR ="40"


import pandas as pd
  
# load the data with pd.read_csv
df = pd.read_csv('munis_v3.csv').head(N) 
print(df)
for index, row in df.iterrows():
    
    if row['muni_name']=='NECOCLI':
       

        print(row['name'], row['REGION'])

        PROVINCE =  row['name']
        MODELO_AREA = row['REGION']
        MUNICIPIO = row['muni_name']

        for YEAR in YEARS:


            try:
                print ("--------------------------------")
                print ("working with ...",MUNICIPIO)
                print ("working with ...",PROVINCE)
                print ("working with ...",MODELO_AREA)
                print ("working with ...",YEAR)
                print ("--------------------------------")
                #satproc
                dataset_folder = execute_satproc_function(SIZE,STEP_SIZE,BASE_PATH,YEAR,PROVINCE,MUNICIPIO)

                #prediction
                excetute_prediction (dataset_folder,BASE_PATH,MODELO_AREA,PROVINCE,MUNICIPIO,YEAR,SIZE,STEP_SIZE)

            except:
                print ("##################################")
                print (f"#     ERR with {PROVINCE}       #")
                print (f"#     ERR with {MUNICIPIO}      #")
                print (f"#     ERR with {YEAR}           #")
                print ("##################################")



        image_1D   =f"{BASE_PATH}/mountain_masks/{PROVINCE}/mask_lt_25_.tif"
        print(image_1D)
        image_1D_resampled   =  image_1D[:-4] + '_resampled.tif'


        vrt_path_2017 = f"{BASE_PATH}/dataset_{MODELO_AREA}/data_results/province_{PROVINCE}_{MUNICIPIO}/2017/all.vrt"
        tiff_path_2017 = f"{BASE_PATH}/dataset_{MODELO_AREA}/data_results/province_{PROVINCE}_{MUNICIPIO}/2017/all.tif"

        !gdal_translate -of GTiff -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 $vrt_path_2017 $tiff_path_2017



        tif_path_2017_resampled = f"{BASE_PATH}/dataset_{MODELO_AREA}/data_results/province_{PROVINCE}_{MUNICIPIO}/2017/all_rs.tif"
        vrt_path_2021 = f"{BASE_PATH}/dataset_{MODELO_AREA}/data_results/province_{PROVINCE}_{MUNICIPIO}/2021/all.vrt"
        thr_tif_file = os.path.join(BASE_PATH,f'dataset_{MODELO_AREA}/data_results/',f'province_{PROVINCE}_{MUNICIPIO}',f'Prob_thr{THR}_province_{PROVINCE}_{MUNICIPIO}_{str(SIZE)}_{str(STEP_SIZE)}_woutmontains_prob.tif') # ruta de destino para nuestra predicción


        subprocess.run(f"bash -c 'source /opt/OTB/otbenv.profile; otbcli_Superimpose -inr {vrt_path_2021} -inm {tiff_path_2017} -out {tif_path_2017_resampled}' ", shell=True)

        #resample
        subprocess.run(f"bash -c 'source /opt/OTB/otbenv.profile; otbcli_Superimpose -inr {vrt_path_2021} -inm {image_1D} -out {image_1D_resampled}' ", shell=True)

        exp = f"(((1-(A/255))*B*(A>{THR})*(B>{THR}))*C)"

        cmd_calc = f'gdal_calc.py -A {tif_path_2017_resampled} --A_band=1  -B {vrt_path_2021} --B_band=1 -C {image_1D_resampled} --C_band=1 --outfile {thr_tif_file} --calc="{exp}" --NoDataValue=0'
        subprocess.run(cmd_calc, shell=True)

        print("saving results: ",thr_tif_file)

        BUCKET_RESULTADOS = f'gs://dym-indunor-temp/immap/v3/results_test/resultados_entrega/{MODELO_AREA}/{PROVINCE}/{MUNICIPIO}/'

        !gsutil cp -r $thr_tif_file $BUCKET_RESULTADOS


    PRIORIDAD            name        muni_name     REGION
0           1       atlantico     BARRANQUILLA  atlantico
1           2       atlantico  PUERTO_COLOMBIA  atlantico
2           3       atlantico          SOLEDAD  atlantico
3           4       atlantico          BARANOA  atlantico
4           5       atlantico      SABANALARGA  atlantico
..        ...             ...              ...        ...
73         74  valle_de_cauca            TULUA   pacifico
74         75  valle_de_cauca           YOTOCO   pacifico
75         76  valle_de_cauca            YUMBO   pacifico
76         77  valle_de_cauca           ZARZAL   pacifico
77         99       ANTIOQUIA          NECOCLI  atlantico

[78 rows x 4 columns]
ANTIOQUIA atlantico
--------------------------------
working with ... NECOCLI
working with ... ANTIOQUIA
working with ... atlantico
working with ... 2017
--------------------------------
path_to_files--> ..//imagenes_s1_atlantico/2017/ANTIOQUIA/*.tif
dataset_folder_--> ..//dataset

2022-06-02 16:54:49.246698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-02 16:54:49.247432: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/cv2/../../lib64:
2022-06-02 16:54:49.247550: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/cv2/../../lib64:
2022-06-02 16:54:49.247631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'

Done!
size 100
step size 100


../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/*tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__41_25.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/all.vrt [Content-Type=application/octet-stream]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__23_11.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__32_28.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__8_27.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands

Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__11_30.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__26_29.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__15_13.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__19_17.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__9_31.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2017/100_100/VHd_VVd_VHdVVd_bands_2017__16_11.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NEC

...

Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__34_46.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__31_48.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__34_22.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__16_6.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__32_49.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__33_48.tif [Content-Type=image/tiff]...
Copying file://../dataset_atl

Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__25_14.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__34_52.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__21_32.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__25_37.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__29_48.tif [Content-Type=image/tiff]...
Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/2021/100_100_joined/VHd_VVd_VHdVVd_bands_2021__26_26.tif [Content-Type=image/tiff]...
| [2.8k/2.8k files][ 28.6 Mi

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 

66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 66.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 67.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 68.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 69.. 70.. 70.. 70.. 70.. 70.. 

Copying file://../dataset_atlantico/data_results/province_ANTIOQUIA_NECOCLI/Prob_thr40_province_ANTIOQUIA_NECOCLI_100_100_woutmontains_prob.tif [Content-Type=image/tiff]...
- [1 files][111.2 MiB/111.2 MiB]                                                
Operation completed over 1 objects/111.2 MiB.                                    
